<a href="https://colab.research.google.com/github/jgbrenner/psychometrics/blob/main/PsychometricsC9_BTPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary Python packages
!pip install openai rpy2

In [2]:
# Import necessary Python libraries
import os
import openai
import pandas as pd
import numpy as np
import json
import requests
import re
from google.colab import drive, userdata

In [3]:
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# Define the path where the R library will be saved in Google Drive
library_path = '/content/drive/MyDrive/R_libraries'

# Create the directory if it doesn't exist
if not os.path.exists(library_path):
    os.makedirs(library_path)

In [5]:
# Verify rpy2 version
import rpy2
print(f"rpy2 version: {rpy2.__version__}")


rpy2 version: 3.4.2


In [7]:


# Set up R environment
from rpy2.robjects import r

# Set the library path in R
r(f'''
library_path <- "{library_path}"
.libPaths(c(library_path, .libPaths()))
''')

# Install and load required R packages
required_packages = [
    "reticulate",
    "dplyr",
    "stringr",
    "readr",
    "EGAnet"
]

for pkg in required_packages:
    r(f'''
    if (!requireNamespace("{pkg}", quietly = TRUE)) {{
        install.packages("{pkg}", lib = "{library_path}", repos = "https://cran.rstudio.com/", dependencies = TRUE)
    }}
    library("{pkg}", character.only = TRUE)
    ''')


EGAnet (version 2.1.0) 

For help getting started, see <https://r-ega.net> 

For bugs and errors, submit an issue to <https://github.com/hfgolino/EGAnet/issues>



In [8]:
# Fetching the OpenAI API key from Colab secrets
openai_api_key = userdata.get('OPENAI_API_KEY')
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
else:
    raise ValueError("OPENAI_API_KEY is not set or invalid.")

# Import OpenAI Client
from openai import Client
client = Client()

# 1. **Generate Scale Items Using OpenAI API**

In [9]:
messages = [
    {
        "role": "system",
        "content": "Jesteś ekspertem w psychometrii, który tworzy pytania testowe."
    },
    {
        "role": "user",
        "content": (
            "Wygeneruj po 3 stwierdzenia dla każdego z następujących konstruktów: "
            "Perfekcjonizm skierowany na siebie: Tendencja do wymagania doskonałości od siebie samego, "
            "Perfekcjonizm skierowany na innych: Stawianie wysokich oczekiwań wobec innych i krytyczna ocena ich osiągnięć, "
            "Perfekcjonizm społecznie narzucony: Przekonanie, że inni oczekują od nas doskonałości. "
            "Niech stwierdzenia będą zwięzłe i jasne,  odpowiednie do oceny w skali Likerta. "
            "Provide the output ONLY in JSON format as a list of dictionaries, "
            "without any additional text or explanation. "
            "Each dictionary should have keys 'construct' and 'item'."
        )
    }
]

try:
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7,
        max_tokens=8048,
        top_p=1,
        stream=False
    )
    response_content = completion.choices[0].message.content
    print("\nGenerated Items:\n", response_content)
except Exception as e:
    print(f"An error occurred: {e}")
    response_content = ""


Generated Items:
 ```json
[
    {"construct": "Perfekcjonizm skierowany na siebie", "item": "Czuję, że muszę być doskonały w każdej dziedzinie swojego życia."},
    {"construct": "Perfekcjonizm skierowany na siebie", "item": "Często czuję się niezadowolony z własnych osiągnięć."},
    {"construct": "Perfekcjonizm skierowany na siebie", "item": "Uważam, że moje błędy są nie do przyjęcia."},
    {"construct": "Perfekcjonizm skierowany na innych", "item": "Oczekuję od innych, że będą osiągać najwyższe wyniki."},
    {"construct": "Perfekcjonizm skierowany na innych", "item": "Często krytycznie oceniam osiągnięcia innych ludzi."},
    {"construct": "Perfekcjonizm skierowany na innych", "item": "Uważam, że inni powinni dążyć do perfekcji tak samo jak ja."},
    {"construct": "Perfekcjonizm społecznie narzucony", "item": "Czuję presję, aby spełniać oczekiwania innych dotyczące doskonałości."},
    {"construct": "Perfekcjonizm społecznie narzucony", "item": "Myślę, że inni oczekują ode mnie 

# 2. **Parse and Validate Data with Fallback**

In [10]:
try:
    # Attempt to parse the response content directly
    generated_items = json.loads(response_content)
    items_df = pd.DataFrame(generated_items)
except json.JSONDecodeError as e:
    print(f"JSON parsing failed: {e}")
    # Attempt to extract JSON-like content using regex
    json_match = re.search(r'\[.*\]', response_content, re.DOTALL)  # Match JSON array
    if json_match:
        json_str = json_match.group(0)  # Extract the matched JSON-like content
        print("Extracted JSON String:")
        print(json_str)
        try:
            # Attempt to parse the extracted string as JSON
            generated_items = json.loads(json_str)
            items_df = pd.DataFrame(generated_items)
        except json.JSONDecodeError as e2:
            print(f"Second JSON decoding attempt failed: {e2}")
            items_df = None
    else:
        print("No valid JSON found in the response.")
        items_df = None

JSON parsing failed: Expecting value: line 1 column 1 (char 0)
Extracted JSON String:
[
    {"construct": "Perfekcjonizm skierowany na siebie", "item": "Czuję, że muszę być doskonały w każdej dziedzinie swojego życia."},
    {"construct": "Perfekcjonizm skierowany na siebie", "item": "Często czuję się niezadowolony z własnych osiągnięć."},
    {"construct": "Perfekcjonizm skierowany na siebie", "item": "Uważam, że moje błędy są nie do przyjęcia."},
    {"construct": "Perfekcjonizm skierowany na innych", "item": "Oczekuję od innych, że będą osiągać najwyższe wyniki."},
    {"construct": "Perfekcjonizm skierowany na innych", "item": "Często krytycznie oceniam osiągnięcia innych ludzi."},
    {"construct": "Perfekcjonizm skierowany na innych", "item": "Uważam, że inni powinni dążyć do perfekcji tak samo jak ja."},
    {"construct": "Perfekcjonizm społecznie narzucony", "item": "Czuję presję, aby spełniać oczekiwania innych dotyczące doskonałości."},
    {"construct": "Perfekcjonizm społec

In [11]:
# Check if the DataFrame was successfully created
if items_df is not None:
    print("\nInitial Items DataFrame:")
    print(items_df.head())
else:
    raise ValueError("Failed to parse JSON. Ensure the LLM response is valid.")


Initial Items DataFrame:
                            construct  \
0  Perfekcjonizm skierowany na siebie   
1  Perfekcjonizm skierowany na siebie   
2  Perfekcjonizm skierowany na siebie   
3  Perfekcjonizm skierowany na innych   
4  Perfekcjonizm skierowany na innych   

                                                item  
0  Czuję, że muszę być doskonały w każdej dziedzi...  
1  Często czuję się niezadowolony z własnych osią...  
2         Uważam, że moje błędy są nie do przyjęcia.  
3  Oczekuję od innych, że będą osiągać najwyższe ...  
4  Często krytycznie oceniam osiągnięcia innych l...  


In [12]:
# Remove duplicates and empty rows
items_df.drop_duplicates(inplace=True)
items_df.dropna(inplace=True)
items_df.reset_index(drop=True, inplace=True)

# Map constructs to abbreviations
construct_abbreviations = {
    "Perfekcjonizm skierowany na siebie": "PSS",
    "Perfekcjonizm skierowany na innych": "PSI",
    "Perfekcjonizm społecznie narzucony": "PSP"
}

# Create descriptive unique labels for items
items_df['item_label'] = items_df.apply(
    lambda row: f"{construct_abbreviations[row['construct']]}_{row.name+1}", axis=1
)

# Inspect the cleaned DataFrame with improved labels
print("\nCleaned Items DataFrame with Improved Labels:")
print(items_df.head())


Cleaned Items DataFrame with Improved Labels:
                            construct  \
0  Perfekcjonizm skierowany na siebie   
1  Perfekcjonizm skierowany na siebie   
2  Perfekcjonizm skierowany na siebie   
3  Perfekcjonizm skierowany na innych   
4  Perfekcjonizm skierowany na innych   

                                                item item_label  
0  Czuję, że muszę być doskonały w każdej dziedzi...      PSS_1  
1  Często czuję się niezadowolony z własnych osią...      PSS_2  
2         Uważam, że moje błędy są nie do przyjęcia.      PSS_3  
3  Oczekuję od innych, że będą osiągać najwyższe ...      PSI_4  
4  Często krytycznie oceniam osiągnięcia innych l...      PSI_5  


In [13]:
# Extract item texts for embedding
item_texts = items_df['item'].tolist()

# Set up the API endpoint and headers for OpenAI embeddings
embedding_endpoint = "https://api.openai.com/v1/embeddings"
embedding_model = "text-embedding-3-small"

headers = {
    "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
    "Content-Type": "application/json"
}

# Prepare the data payload
data = {
    "model": embedding_model,
    "input": item_texts
}

try:
    response = requests.post(embedding_endpoint, headers=headers, json=data)
    if response.status_code == 200:
        response_data = response.json()
        embeddings = [item['embedding'] for item in response_data['data']]
        embeddings_array = np.array(embeddings)
        print(f"Embeddings generated successfully. Shape: {embeddings_array.shape}")
    else:
        print(f"Request failed with status code {response.status_code}: {response.text}")
        embeddings_array = None
except Exception as e:
    print(f"An error occurred during embedding generation: {e}")
    embeddings_array = None

# Check alignment between items and embeddings
if embeddings_array is not None and embeddings_array.shape[0] != len(items_df):
    print(f"Embeddings count ({embeddings_array.shape[0]}) does not match items count ({len(items_df)}). Adjusting items_df.")
    items_df = items_df.iloc[:embeddings_array.shape[0]]
    items_df.reset_index(drop=True, inplace=True)

# Add embeddings back to DataFrame for inspection
items_df['embedding'] = list(embeddings_array)
print("\nItems DataFrame with Embeddings:")
print(items_df.head())

# Save DataFrame as CSV for inspection in R
items_df.to_csv("psychometric_items_with_embeddings.csv", index=False)

# Save embeddings as a CSV file
np.savetxt("embeddings.csv", embeddings_array, delimiter=",")
print("Embeddings saved as CSV.")

# Inspect saved file locations and sizes
print("\nSaved files:")
print("Items DataFrame (CSV):", os.path.abspath("psychometric_items_with_embeddings.csv"))
print("Embeddings (CSV):", os.path.abspath("embeddings.csv"))

Embeddings generated successfully. Shape: (9, 1536)

Items DataFrame with Embeddings:
                            construct  \
0  Perfekcjonizm skierowany na siebie   
1  Perfekcjonizm skierowany na siebie   
2  Perfekcjonizm skierowany na siebie   
3  Perfekcjonizm skierowany na innych   
4  Perfekcjonizm skierowany na innych   

                                                item item_label  \
0  Czuję, że muszę być doskonały w każdej dziedzi...      PSS_1   
1  Często czuję się niezadowolony z własnych osią...      PSS_2   
2         Uważam, że moje błędy są nie do przyjęcia.      PSS_3   
3  Oczekuję od innych, że będą osiągać najwyższe ...      PSI_4   
4  Często krytycznie oceniam osiągnięcia innych l...      PSI_5   

                                           embedding  
0  [0.05987178, 0.022176193, -0.031097876, 0.0631...  
1  [0.0336493, 0.018436313, -0.025339449, 0.02029...  
2  [0.037936278, 0.035252683, -0.0049656676, 0.05...  
3  [0.056001738, 0.006498609, 0.0067103994, 

Load rpy2

In [15]:
%load_ext rpy2.ipython

In [16]:
%%R

# Load required libraries and set library path
library_path <- '/content/drive/MyDrive/R_libraries'
.libPaths(library_path)

library(reticulate, lib.loc = library_path)
library(dplyr, lib.loc = library_path)
library(stringr, lib.loc = library_path)
library(readr, lib.loc = library_path)
library(EGAnet, lib.loc = library_path)

# Load psychometric items and embeddings
items_df <- read.csv("psychometric_items_with_embeddings.csv")

# Load the CSV file for embeddings
embeddings_array <- as.matrix(read.csv("embeddings.csv", header = FALSE))

# Inspect loaded data
print("Loaded Items DataFrame:")
print(head(items_df))

print("Loaded Embeddings Array Dimensions:")
print(dim(embeddings_array))


[1] "Loaded Items DataFrame:"
                           construct
1 Perfekcjonizm skierowany na siebie
2 Perfekcjonizm skierowany na siebie
3 Perfekcjonizm skierowany na siebie
4 Perfekcjonizm skierowany na innych
5 Perfekcjonizm skierowany na innych
6 Perfekcjonizm skierowany na innych
                                                              item item_label
1 Czuję, że muszę być doskonały w każdej dziedzinie swojego życia.      PSS_1
2             Często czuję się niezadowolony z własnych osiągnięć.      PSS_2
3                       Uważam, że moje błędy są nie do przyjęcia.      PSS_3
4            Oczekuję od innych, że będą osiągać najwyższe wyniki.      PSI_4
5              Często krytycznie oceniam osiągnięcia innych ludzi.      PSI_5
6      Uważam, że inni powinni dążyć do perfekcji tak samo jak ja.      PSI_6
                                                                        embedding
1 [ 0.05987178  0.02217619 -0.03109788 ... -0.00722302 -0.00905955\n -0.02231405]
2

In [22]:
%%R

# Compute the correlation matrix from embeddings
cor_matrix <- cor(t(embeddings_array))
cor_matrix[is.na(cor_matrix)] <- 0  # Replace NA values with 0

# Assign item labels as row and column names
rownames(cor_matrix) <- items_df$item_label
colnames(cor_matrix) <- items_df$item_label

# Apply Unique Variable Analysis (UVA) to identify redundant items
uva_result <- UVA(
    data = cor_matrix,
    n = nrow(items_df),
    method = "wTO",  # Weighted Topological Overlap
    threshold = 0.20  # Adjust threshold as needed
)

# Print redundant items
redundant_items <- uva_result$redundant
print("Redundant Items Identified:")
print(redundant_items)

# Filter top 6 items for each construct
top_items <- items_df[!items_df$item_label %in% redundant_items, ] %>%
  group_by(construct) %>%
  slice_head(n = 6)  # Select top 6 items per construct
print("Top 6 Items Per Construct:")
print(top_items)

[1] "Redundant Items Identified:"
NULL
[1] "Top 6 Items Per Construct:"
# A tibble: 9 × 4
# Groups:   construct [3]
  construct                          item                   item_label embedding
  <chr>                              <chr>                  <chr>      <chr>    
1 Perfekcjonizm skierowany na innych Oczekuję od innych, ż… PSI_4      "[ 0.056…
2 Perfekcjonizm skierowany na innych Często krytycznie oce… PSI_5      "[ 0.046…
3 Perfekcjonizm skierowany na innych Uważam, że inni powin… PSI_6      "[ 0.027…
4 Perfekcjonizm skierowany na siebie Czuję, że muszę być d… PSS_1      "[ 0.059…
5 Perfekcjonizm skierowany na siebie Często czuję się niez… PSS_2      "[ 0.033…
6 Perfekcjonizm skierowany na siebie Uważam, że moje błędy… PSS_3      "[ 0.037…
7 Perfekcjonizm społecznie narzucony Czuję presję, aby spe… PSP_7      "[ 0.048…
8 Perfekcjonizm społecznie narzucony Myślę, że inni oczeku… PSP_8      "[ 0.016…
9 Perfekcjonizm społecznie narzucony Obawiam się, że nie b… PSP_9      "[ 

In [20]:
%%R

# Remove redundant items from the embeddings and correlation matrix
if (!is.null(redundant_items) && length(redundant_items) > 0) {
    embeddings_array <- embeddings_array[!(rownames(embeddings_array) %in% redundant_items), ]
    cor_matrix <- cor_matrix[!(rownames(cor_matrix) %in% redundant_items), !(colnames(cor_matrix) %in% redundant_items)]
    items_df <- items_df[!(items_df$item_label %in% redundant_items), ]
}

# Recompute the correlation matrix after removing redundant items
cor_matrix <- cor(t(embeddings_array))
cor_matrix[is.na(cor_matrix)] <- 0  # Replace NA values with 0

# Assign item labels as row and column names again
rownames(cor_matrix) <- items_df$item_label
colnames(cor_matrix) <- items_df$item_label

# Verify alignment
print("Aligned Correlation Matrix Dimensions:")
print(dim(cor_matrix))

print("Correlation Matrix Row Names:")
print(rownames(cor_matrix)[1:5])

print("Correlation Matrix Column Names:")
print(colnames(cor_matrix)[1:5])

[1] "Aligned Correlation Matrix Dimensions:"
[1] 9 9
[1] "Correlation Matrix Row Names:"
[1] "PSS_1" "PSS_2" "PSS_3" "PSI_4" "PSI_5"
[1] "Correlation Matrix Column Names:"
[1] "PSS_1" "PSS_2" "PSS_3" "PSI_4" "PSI_5"


In [23]:
%%R

# Perform EGA to detect latent constructs
ega_result <- EGA(
    data = cor_matrix,
    model = "glasso",    # Graphical Lasso
    algorithm = "walktrap",  # Community detection algorithm
    n = nrow(items_df)   # Sample size
)

# Inspect EGA results
print("EGA Communities Detected:")
print(ega_result$wc)  # Community memberships for each item

# Plot EGA results
plot(ega_result, layout = "spring")

[1] "EGA Communities Detected:"
Algorithm:  Louvain

Number of communities:  1

PSS_1 PSS_2 PSS_3 PSI_4 PSI_5 PSI_6 PSP_7 PSP_8 PSP_9 
    1     1     1     1     1     1     1     1     1 


NULL


In [25]:
%%R

# Save EGA graph
pdf("EGA_graph.pdf")
plot(ega_result, layout = "spring")
dev.off()

# Save top items to CSV
write.csv(top_items, "top_6_items_per_construct.csv", row.names = FALSE)